In [1]:
import numpy as np
import pandas as pd
from itertools import product
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
import random
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error

2024-12-17 11:11:17.390711: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-17 11:11:18.048606: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-17 11:11:18.288715: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-17 11:11:18.360670: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-17 11:11:18.860775: I tensorflow/core/platform/cpu_feature_guar

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.config.list_physical_devices()

Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# List all physical devices
gpus = tf.config.list_physical_devices('GPU')
print("Available GPUs:")
for gpu in gpus:
    print(f" - {gpu.name}")

Available GPUs:
 - /physical_device:GPU:0


In [4]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth enabled for GPUs.")
    except RuntimeError as e:
        print(e)

Memory growth enabled for GPUs.


In [5]:
traffic_data = pd.read_csv('GD030A_S.csv')

## 1. Recover timestamp

In [6]:
# Define the recover_timestamp function
def recover_timestamp(data):
    # Combine 'date' and 'time' to form a datetime column
    data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['time'].astype(str) + ':00', format='%Y-%m-%d %H:%M')

    # Set 'datetime' as index
    data = data.set_index('datetime')

    # Create a complete range of timestamps with hourly frequency
    full_time_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='H')

    # Reindex the data to include all timestamps, filling missing rows with NaN
    data_full = data.reindex(full_time_range)

    return data_full

In [7]:
# Apply the recover_timestamp function to recover the full time series
traffic_full = recover_timestamp(traffic_data)
traffic_full

,date,time,flow
2019-10-01 00:00:00,2019-10-01,0.0,15.0
2019-10-01 01:00:00,2019-10-01,1.0,9.0
2019-10-01 02:00:00,2019-10-01,2.0,9.0
2019-10-01 03:00:00,2019-10-01,3.0,7.0
2019-10-01 04:00:00,2019-10-01,4.0,9.0
...,...,...,...
2023-09-30 19:00:00,2023-09-30,19.0,129.0
2023-09-30 20:00:00,2023-09-30,20.0,119.0
2023-09-30 21:00:00,2023-09-30,21.0,106.0
2023-09-30 22:00:00,2023-09-30,22.0,88.0


## 2. Train, validate, test data split

In [8]:
train_set = traffic_full[:'2022-02-28 23:00:00']
valid_set = traffic_full['2022-03-01 00:00:00':'2022-12-31 23:00:00']
test_set = traffic_full['2023-01-01 00:00:00':]
# train_set = traffic_full['2022-06-03 00:00:00':'2023-03-31 23:00:00']
# valid_set = traffic_full['2023-04-01 00:00:00':'2023-06-30 23:00:00']
# test_set = traffic_full['2023-07-01 00:00:00':]
print('Proportion of train_set : {:.4f}'.format(len(train_set)/len(traffic_full['2022-06-03 00:00:00':])))
print('Proportion of valid_set : {:.4f}'.format(len(valid_set)/len(traffic_full['2022-06-03 00:00:00':])))
print('Proportion of test_set : {:.4f}'.format(len(test_set)/len(traffic_full['2022-06-03 00:00:00':])))

Proportion of train_set : 1.8186
Proportion of valid_set : 0.6309
Proportion of test_set : 0.5629


In [9]:
print(train_set.isnull().sum(), len(train_set))
print(valid_set.isnull().sum(),len(valid_set))
print(test_set.isnull().sum(),len(test_set))

date    927
time    927
flow    927
dtype: int64 21168
date    91
time    91
flow    91
dtype: int64 7344
date    403
time    403
flow    403
dtype: int64 6552


## 3. Normalise the data 

In [10]:
# Initialize the scaler
scaler = MinMaxScaler()

# Fit the scaler on the training data's 'flow' feature
scaler.fit(train_set[['flow']])

# Transform the 'flow' feature in all datasets
train_set.loc[:, 'flow_scaled'] = scaler.transform(train_set[['flow']])
valid_set.loc[:, 'flow_scaled'] = scaler.transform(valid_set[['flow']])
test_set.loc[:, 'flow_scaled'] = scaler.transform(test_set[['flow']])

/tmp/ipykernel_539389/1100395100.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set.loc[:, 'flow_scaled'] = scaler.transform(train_set[['flow']])
/tmp/ipykernel_539389/1100395100.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_set.loc[:, 'flow_scaled'] = scaler.transform(valid_set[['flow']])
/tmp/ipykernel_539389/1100395100.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

## 4. Split the data into X and y

In [11]:
def create_sequences(data, input_length, forecast_horizon):
    """
    Creates input-output sequences for time series data, excluding any sequences containing NaN values.
    
    Parameters:
    - data: pandas DataFrame containing the data. Must include the 'flow_scaled' column.
    - input_length: int, number of past time steps to include in each input sequence.
    - forecast_horizon: int, number of future steps to predict.
    
    Returns:
    - X: numpy array of shape (num_valid_samples, input_length, num_features)
    - y: numpy array of shape (num_valid_samples, forecast_horizon)
    """
    X, y = [], []
    num_features = data.shape[1]
    total_length = input_length + forecast_horizon
    
    for i in range(input_length, len(data) - forecast_horizon + 1):
        # Extract the input sequence
        X_seq = data.iloc[i - input_length:i]['flow_scaled'].values
        # Extract the target sequence
        y_seq = data.iloc[i:i + forecast_horizon]['flow_scaled'].values
        
        # Check for NaN values in the input sequence and target sequence
        if not np.isnan(X_seq).any() and not np.isnan(y_seq).any():
            X.append(X_seq)
            y.append(y_seq)
        else:
            # Optionally, log or count the skipped sequences
            pass  # Simply skip sequences with NaNs
        
    # Convert to numpy arrays and reshape X to match LSTM expected input (samples, timesteps, features)
    X = np.array(X).reshape(-1, input_length, 1)
    y = np.array(y).reshape(-1, forecast_horizon)
    
    return X, y

## 5. Create X and y

#### We will use
* the last 24*N steps

*  to forecast current (0 step) and 5 steps ahead

In [12]:
# Define Input Sequence Lengths
input_lengths = [24 * i for i in range(20, 22)]  # [24, 48, ..., 168]

In [13]:
from collections import defaultdict
data_dict = defaultdict(dict)

for length in input_lengths:
    print(f"Processing input length: {length}")
    
    # Create sequences with forecast_horizon=6
    X_train, y_train = create_sequences(train_set, length, forecast_horizon=6)
    X_val, y_val = create_sequences(valid_set, length, forecast_horizon=6)
    X_test, y_test = create_sequences(test_set, length, forecast_horizon=6)
    
    # Store in the dictionary
    data_dict[length]['X_train'] = X_train
    data_dict[length]['y_train'] = y_train
    data_dict[length]['X_val'] = X_val
    data_dict[length]['y_val'] = y_val
    data_dict[length]['X_test'] = X_test
    data_dict[length]['y_test'] = y_test
    
    # Print shapes and ensure no NaNs
    print(f"  X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
    print(f"  X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
    print(f"  X_test shape: {X_test.shape}, y_test shape: {y_test.shape}\n")

Processing input length: 480
  X_train shape: (13296, 480, 1), y_train shape: (13296, 6)
  X_val shape: (5015, 480, 1), y_val shape: (5015, 6)
  X_test shape: (2883, 480, 1), y_test shape: (2883, 6)

Processing input length: 504
  X_train shape: (13046, 504, 1), y_train shape: (13046, 6)
  X_val shape: (4919, 504, 1), y_val shape: (4919, 6)
  X_test shape: (2781, 504, 1), y_test shape: (2781, 6)



## 6. Build LSTM model

In [14]:
def build_lstm_model(hyperparams, input_length):
    
    model = Sequential()
    model.add(LSTM(units=hyperparams['units'], activation='tanh', input_shape=(input_length, 1)))
    model.add(Dropout(rate=hyperparams['dropout']))
    model.add(Dense(6))  # Output layer for multi-step forecasting

    # Compile the model with MSE as the loss function
    optimizer = keras.optimizers.Adam(learning_rate=hyperparams['learning_rate'])
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])
    
    return model

In [14]:
def build_improved_lstm_model(hyperparams, input_length):
    """
    Builds an improved LSTM model based on provided hyperparameters and input length.

    Parameters:
    - hyperparams: dict containing 'units', 'dropout', 'learning_rate', and optionally 'recurrent_dropout'.
    - input_length: int, length of the input sequences.

    Returns:
    - model: compiled Keras model ready for training.
    """
    model = Sequential()
    
    # First LSTM layer with return_sequences=True to stack another LSTM layer
    model.add(LSTM(
        units=hyperparams['units'], 
        activation='tanh', 
        input_shape=(input_length, 1), 
        return_sequences=True, 
        #recurrent_dropout=hyperparams.get('recurrent_dropout', 0.0)
    ))
    model.add(Dropout(rate=hyperparams['dropout']))
    
    # Second LSTM layer
    model.add(LSTM(
        units=hyperparams['units'], 
        activation='tanh',
        #recurrent_dropout=hyperparams.get('recurrent_dropout', 0.0)
    ))
    model.add(Dropout(rate=hyperparams['dropout']))
    
    # Output layer for multi-step forecasting
    model.add(Dense(6))
    
    # Compile the model with MSE as the loss function
    optimizer = keras.optimizers.Adam(learning_rate=hyperparams['learning_rate'])
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])
    
    return model


## 7. Define the hyperparameter grid

In [15]:
# Hyperparameter options
units_list = [32, 64, 128, 256]
dropout_rates = [0.0, 0.1, 0.2, 0.3, 0.4]
learning_rates = [0.01, 0.005, 0.001]
batch_sizes = [32, 64, 128]

# Create all possible combinations
all_combinations = list(product(units_list, dropout_rates, learning_rates, batch_sizes))

In [16]:
all_combinations

[(32, 0.0, 0.01, 32),
 (32, 0.0, 0.01, 64),
 (32, 0.0, 0.01, 128),
 (32, 0.0, 0.005, 32),
 (32, 0.0, 0.005, 64),
 (32, 0.0, 0.005, 128),
 (32, 0.0, 0.001, 32),
 (32, 0.0, 0.001, 64),
 (32, 0.0, 0.001, 128),
 (32, 0.1, 0.01, 32),
 (32, 0.1, 0.01, 64),
 (32, 0.1, 0.01, 128),
 (32, 0.1, 0.005, 32),
 (32, 0.1, 0.005, 64),
 (32, 0.1, 0.005, 128),
 (32, 0.1, 0.001, 32),
 (32, 0.1, 0.001, 64),
 (32, 0.1, 0.001, 128),
 (32, 0.2, 0.01, 32),
 (32, 0.2, 0.01, 64),
 (32, 0.2, 0.01, 128),
 (32, 0.2, 0.005, 32),
 (32, 0.2, 0.005, 64),
 (32, 0.2, 0.005, 128),
 (32, 0.2, 0.001, 32),
 (32, 0.2, 0.001, 64),
 (32, 0.2, 0.001, 128),
 (32, 0.3, 0.01, 32),
 (32, 0.3, 0.01, 64),
 (32, 0.3, 0.01, 128),
 (32, 0.3, 0.005, 32),
 (32, 0.3, 0.005, 64),
 (32, 0.3, 0.005, 128),
 (32, 0.3, 0.001, 32),
 (32, 0.3, 0.001, 64),
 (32, 0.3, 0.001, 128),
 (32, 0.4, 0.01, 32),
 (32, 0.4, 0.01, 64),
 (32, 0.4, 0.01, 128),
 (32, 0.4, 0.005, 32),
 (32, 0.4, 0.005, 64),
 (32, 0.4, 0.005, 128),
 (32, 0.4, 0.001, 32),
 (32, 0.4, 0

## 8. Defining manual grid search

In [47]:
# creating a log file (do not need to run when log file already created)
logging.basicConfig(
    level=logging.INFO,  # Set the logging level to INFO
    format='%(message)s',  # Customize the log message format
    handlers=[
        logging.FileHandler('lstm.log'),  # Log messages to 'output.log'
        logging.StreamHandler()             # Also output to console/notebook
    ]
)

In [52]:
# Properly close handlers to ensure flushing to the file
for handler in logger.handlers:
    handler.close()
    logger.removeHandler(handler)

In [17]:
import logging

# Create a custom logger
logger = logging.getLogger('my_logger')

if not logger.handlers:
    logger.setLevel(logging.INFO)

    c_handler = logging.StreamHandler()
    f_handler = logging.FileHandler('lstm.log')

    formatter = logging.Formatter('%(asctime)s - %(message)s')
    c_handler.setFormatter(formatter)
    f_handler.setFormatter(formatter)

    logger.addHandler(c_handler)
    logger.addHandler(f_handler)

In [ ]:
# Initialize a list to store the results
results = []

for length in input_lengths:
    logger.info(f"Starting grid search for input length: {length}")
    
    X_train = data_dict[length]['X_train']
    y_train = data_dict[length]['y_train']
    X_val = data_dict[length]['X_val']
    y_val = data_dict[length]['y_val']
    
    best_mse = float('inf')
    best_params = {}
    best_model = None
    
    for idx, combination in enumerate(all_combinations):
        # Extract hyperparameters
        hyperparams = {
            'units': combination[0],
            'dropout': combination[1],
            'learning_rate': combination[2],
            'batch_size': combination[3]
        }
        
        logger.info(f"  Evaluating combination {idx + 1}/{len(all_combinations)}: {hyperparams}")
        
        #model = build_lstm_model(hyperparams, length)
        model = build_improved_lstm_model(hyperparams, length)
        
        # Early Stopping Callback
        early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True,verbose=1)
        
        # Train the model
        history = model.fit(
            X_train, y_train,
            epochs=100,
            batch_size=hyperparams['batch_size'],
            validation_data=(X_val, y_val),
            callbacks=[early_stop],
            verbose=0  # Set to 1 to see training progress
        )
        
        # Retrieve the best validation MSE from the history
        current_best_mse = min(history.history['val_loss'])
        logger.info(f"Validation loss: {current_best_mse:.5f}")
        
        # Check if this is the best model so far
        if current_best_mse < best_mse:
            best_mse = current_best_mse
            best_params = hyperparams.copy()
            best_model = model  # Optionally, save the model if needed
    
    # After evaluating all combinations, store the best results
    results.append({
        'Input_Length': length,
        'Best_MSE': best_mse,
        'Validation_MSE': mean_squared_error(y_val, best_model.predict(X_val)),
        'Validation_MAE': mean_absolute_error(y_val, best_model.predict(X_val)),
        'Validation_MAPE': mean_absolute_percentage_error(y_val, best_model.predict(X_val)) * 100,  # In percentage
        'Validation_RMSE': np.sqrt(mean_squared_error(y_val, best_model.predict(X_val))),
        'Best_Hyperparameters': best_params
    })
    
    logger.info(f"Completed grid search for input length: {length}")
    logger.info(f"  Best Validation MSE: {best_mse:.5f}")
    logger.info(f"  Best Hyperparameters: {best_params}\n")

2024-12-17 11:18:33,892 - Starting grid search for input length: 480
2024-12-17 11:18:33,893 -   Evaluating combination 1/180: {'units': 32, 'dropout': 0.0, 'learning_rate': 0.01, 'batch_size': 32}
2024-12-17 11:18:34.136425: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22263 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 276f:00:00.0, compute capability: 8.0
/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2024-12-17 11:18:37.651045: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


Epoch 35: early stopping
Restoring model weights from the end of the best epoch: 15.


2024-12-17 11:29:45,955 - Validation loss: 0.00607
2024-12-17 11:29:45,957 -   Evaluating combination 2/180: {'units': 32, 'dropout': 0.0, 'learning_rate': 0.01, 'batch_size': 64}


Epoch 29: early stopping
Restoring model weights from the end of the best epoch: 9.


2024-12-17 11:34:25,787 - Validation loss: 0.00608
2024-12-17 11:34:25,790 -   Evaluating combination 3/180: {'units': 32, 'dropout': 0.0, 'learning_rate': 0.01, 'batch_size': 128}


Epoch 47: early stopping
Restoring model weights from the end of the best epoch: 27.


2024-12-17 11:38:18,823 - Validation loss: 0.00601
2024-12-17 11:38:18,825 -   Evaluating combination 4/180: {'units': 32, 'dropout': 0.0, 'learning_rate': 0.005, 'batch_size': 32}


Epoch 33: early stopping
Restoring model weights from the end of the best epoch: 13.


2024-12-17 11:48:46,573 - Validation loss: 0.00605
2024-12-17 11:48:46,574 -   Evaluating combination 5/180: {'units': 32, 'dropout': 0.0, 'learning_rate': 0.005, 'batch_size': 64}


Completed grid search for input length: 24
  Best Validation MSE: 0.00553
  Best Hyperparameters: {'units': 64, 'dropout': 0.2, 'learning_rate': 0.001, 'batch_size': 32}

Completed grid search for input length: 48
  Best Validation MSE: 0.00546
  Best Hyperparameters: {'units': 256, 'dropout': 0.2, 'learning_rate': 0.01, 'batch_size': 32}

Completed grid search for input length: 72
  Best Validation MSE: 0.00531
  Best Hyperparameters: {'units': 256, 'dropout': 0.1, 'learning_rate': 0.001, 'batch_size': 128}

Completed grid search for input length: 96
Best Validation MSE: 0.00535
Best Hyperparameters: {'units': 256, 'dropout': 0.4, 'learning_rate': 0.001, 'batch_size': 32}

Completed grid search for input length: 120 Best Validation MSE: 0.00528 Best Hyperparameters: {'units': 128, 'dropout': 0.1, 'learning_rate': 0.001, 'batch_size': 64}

Completed grid search for input length: 144 Best Validation MSE: 0.00528 Best Hyperparameters: {'units': 256, 'dropout': 0.2, 'learning_rate': 0.001, 'batch_size': 64}

Completed grid search for input length: 168 Best Validation MSE: 0.00526 Best Hyperparameters: {'units': 256, 'dropout': 0.3, 'learning_rate': 0.001, 'batch_size': 64}

Completed grid search for input length: 192 Best Validation MSE: 0.00524 Best Hyperparameters: {'units': 256, 'dropout': 0.4, 'learning_rate': 0.001, 'batch_size': 32}

Completed grid search for input length: 216 Best Validation MSE: 0.00537 Best Hyperparameters: {'units': 256, 'dropout': 0.4, 'learning_rate': 0.001, 'batch_size': 32}


Completed grid search for input length: 240 Best Validation MSE: 0.00541 Best Hyperparameters: {'units': 64, 'dropout': 0.4, 'learning_rate': 0.005, 'batch_size': 64}

Completed grid search for input length: 264 Best Validation MSE: 0.00546 Best Hyperparameters: {'units': 256, 'dropout': 0.4, 'learning_rate': 0.001, 'batch_size': 32}

Completed grid search for input length: 288 Best Validation MSE: 0.00547 Best Hyperparameters: {'units': 64, 'dropout': 0.3, 'learning_rate': 0.001, 'batch_size': 64}

Completed grid search for input length: 312
Best Validation MSE: 0.00555
Best Hyperparameters: {'units': 32, 'dropout': 0.2, 'learning_rate': 0.001, 'batch_size': 32}

Completed grid search for input length: 336
Best Validation MSE: 0.00556
Best Hyperparameters: {'units': 256, 'dropout': 0.1, 'learning_rate': 0.01, 'batch_size': 64}

Completed grid search for input length: 360
Best Validation MSE: 0.00570
Best Hyperparameters: {'units': 32, 'dropout': 0.1, 'learning_rate': 0.001, 'batch_size': 32}

Completed grid search for input length: 384
Best Validation MSE: 0.00574
Best Hyperparameters: {'units': 256, 'dropout': 0.3, 'learning_rate': 0.001, 'batch_size': 64}

Completed grid search for input length: 408
Best Validation MSE: 0.00565
Best Hyperparameters: {'units': 256, 'dropout': 0.4, 'learning_rate': 0.001, 'batch_size': 64}


Completed grid search for input length: 432
Best Validation MSE: 0.00568
Best Hyperparameters: {'units': 128, 'dropout': 0.4, 'learning_rate': 0.001, 'batch_size': 64}


Completed grid search for input length: 456
Best Validation MSE: 0.00572
Best Hyperparameters: {'units': 128, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 32}

## 8. Storing Results

In [ ]:
# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

# Expand the hyperparameters dictionary into separate columns for clarity
hyperparams_df = results_df['Best_Hyperparameters'].apply(pd.Series)

# Combine the main dataframe with hyperparameters
final_results_df = pd.concat([results_df.drop('Best_Hyperparameters', axis=1), hyperparams_df], axis=1)

# Display the final dataframe
print("Final Results DataFrame:")
final_results_df

## 9. Retrain the model after getting the best hyperparameters of each input length

In [25]:
best_hyperparameters = {
    24: {
        'units': 128,
        'dropout': 0.1,
        'learning_rate': 0.0010,
        'batch_size': 32
    },
    48: {
        'units': 128,
        'dropout': 0.1,
        'learning_rate': 0.0010,
        'batch_size': 32
    },
    72: {
        'units': 128,
        'dropout': 0.2,
        'learning_rate': 0.0100,
        'batch_size': 128
    },
    96: {
        'units': 32,
        'dropout': 0.2,
        'learning_rate': 0.0100,
        'batch_size': 32
    },
    120: {
        'units': 128,
        'dropout': 0.1,
        'learning_rate': 0.0010,
        'batch_size': 32
    },
    144: {
        'units': 256,
        'dropout': 0.0,
        'learning_rate': 0.0100,
        'batch_size': 64
    },
    168: {
        'units': 128,
        'dropout': 0.1,
        'learning_rate': 0.0100,
        'batch_size': 128
    },
    192: {
        'units': 64,
        'dropout': 0.0,
        'learning_rate': 0.0005,
        'batch_size': 64
    },
    216: {
        'units': 32,
        'dropout': 0.2,
        'learning_rate': 0.0100,
        'batch_size': 32
    },
    240: {
        'units': 128,
        'dropout': 0.3,
        'learning_rate': 0.0100,
        'batch_size': 32
    },
    264: {
        'units': 64,
        'dropout': 0.2,
        'learning_rate': 0.0100,
        'batch_size': 64
    },
    288: {
        'units': 32,
        'dropout': 0.2,
        'learning_rate': 0.0100,
        'batch_size': 32
    },
    312: {
        'units': 256,
        'dropout': 0.3,
        'learning_rate': 0.0100,
        'batch_size': 32
    },
    336: {
        'units': 64,
        'dropout': 0.3,
        'learning_rate': 0.0100,
        'batch_size': 32
    },
    360: {
        'units': 64,
        'dropout': 0.0,
        'learning_rate': 0.0100,
        'batch_size': 128
    },
    384: {
        'units': 128,
        'dropout': 0.2,
        'learning_rate': 0.0100,
        'batch_size': 32
    },
    408: {
        'units': 64,
        'dropout': 0.1,
        'learning_rate': 0.0100,
        'batch_size': 32
    },
    432: {
        'units': 256,
        'dropout': 0.0,
        'learning_rate': 0.0100,
        'batch_size': 64
    },
    456: {
        'units': 32,
        'dropout': 0.3,
        'learning_rate': 0.0100,
        'batch_size': 32
    },
    480: {
        'units': 128,
        'dropout': 0.4,
        'learning_rate': 0.0100,
        'batch_size': 32
    },
    504: {
        'units': 64,
        'dropout': 0.3,
        'learning_rate': 0.0100,
        'batch_size': 32
    }
}


In [ ]:
best_hyperparameters_improved_model = {
    24: {
        'units': 32,
        'dropout': 0.2,
        'learning_rate': 0.001,
        'batch_size': 32
    },
    48: {
        'units': 128,
        'dropout': 0.2,
        'learning_rate': 0.01,
        'batch_size': 64
    },
    72: {
        'units': 256,
        'dropout': 0.0,
        'learning_rate': 0.001,
        'batch_size': 32
    },
    96: {
        'units': 64,
        'dropout': 0.2,
        'learning_rate': 0.01,
        'batch_size': 32
    },
    120: {
        'units': 32,
        'dropout': 0.2,
        'learning_rate': 0.001,
        'batch_size': 32
    },
    144: {
        'units': 128,
        'dropout': 0.3,
        'learning_rate': 0.01,
        'batch_size': 32
    },
    168: {
        'units': 32,
        'dropout': 0.0,
        'learning_rate': 0.0005,
        'batch_size': 32
    },
    192: {
        'units': 64,
        'dropout': 0.2,
        'learning_rate': 0.01,
        'batch_size': 128
    },
    216: {
        'units': 128,
        'dropout': 0.1,
        'learning_rate': 0.01,
        'batch_size': 32
    },
    240: {
        'units': 128,
        'dropout': 0.1,
        'learning_rate': 0.01,
        'batch_size': 32
    },
    264: {
        'units': 128,
        'dropout': 0.3,
        'learning_rate': 0.01,
        'batch_size': 32
    },
    288: {
        'units': 128,
        'dropout': 0.0,
        'learning_rate': 0.01,
        'batch_size': 32
    },
    312: {
        'units': 64,
        'dropout': 0.2,
        'learning_rate': 0.01,
        'batch_size': 64
    },
    336: {
        'units': 128,
        'dropout': 0.3,
        'learning_rate': 0.01,
        'batch_size': 32
    },
    360: {
        'units': 128,
        'dropout': 0.2,
        'learning_rate': 0.001,
        'batch_size': 32
    },
    384: {
        'units': 64,
        'dropout': 0.2,
        'learning_rate': 0.01,
        'batch_size': 32
    },
    408: {
        'units': 128,
        'dropout': 0.2,
        'learning_rate': 0.01,
        'batch_size': 32
    },
    432: {
        'units': 64,
        'dropout': 0.3,
        'learning_rate': 0.01,
        'batch_size': 32
    },
    456: {
        'units': 64,
        'dropout': 0.3,
        'learning_rate': 0.01,
        'batch_size': 32
    },
    480: {
        'units': 32,
        'dropout': 0.4,
        'learning_rate': 0.01,
        'batch_size': 32
    },
    504: {
        'units': 128,
        'dropout': 0.4,
        'learning_rate': 0.01,
        'batch_size': 32
    }
}


In [18]:
def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)

In [27]:
def train_model(hyperparams,data_dict,length, seed=None):  # add seed
    if seed is not None:
        set_seed(seed)
    
    #get the data of each length
    X_train = data_dict[length]['X_train']
    y_train = data_dict[length]['y_train']
    X_val = data_dict[length]['X_val']
    y_val = data_dict[length]['y_val']
    
    # Train the model
    #model = build_lstm_model(hyperparams, length)    
    model = build_improved_lstm_model(hyperparams, length)
    # Early Stopping Callback
    early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True,verbose=1)    
    
    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=hyperparams['batch_size'],
        validation_data=(X_val, y_val),
        callbacks=[early_stop],
        verbose=0  # Set to 1 to see training progress
    )
    
    # Retrieve the best validation MSE from the history
    best_mse = min(history.history['val_loss'])
    logger.info(f"Validation loss: {best_mse:.5f}")
    
    return model, best_mse

In [20]:
# Make predictions
def make_prediction(model, X_obs, y_obs):
    y_pred = model.predict(X_obs,verbose=0)
    n_samples = X_obs.shape[0]
    output_len = y_obs.shape[1]

    # Reshape for inverse scaling
    y_pred_reshaped = y_pred.reshape(-1, 1)
    y_obs_reshaped = y_obs.reshape(-1, 1)

    # Inverse transform
    y_pred_inverse = scaler.inverse_transform(y_pred_reshaped).reshape(n_samples, output_len)
    y_obs_inverse = scaler.inverse_transform(y_obs_reshaped).reshape(n_samples, output_len)

    return y_pred_inverse, y_obs_inverse

In [21]:
# Compute Metrics for Each Time Step
def evaluation(y_pred_inverse, y_obs_inverse):
    
    output_len = y_pred_inverse.shape[1]
    metrics_list = []  # To store metrics for each time step
    
    for i in range(output_len):
        y_true = y_obs_inverse[:, i]
        y_pred = y_pred_inverse[:, i]

        # Mean Absolute Error (MAE)
        mae = mean_absolute_error(y_true, y_pred)

        # Mean Squared Error (MSE)
        mse = mean_squared_error(y_true, y_pred)

        # Root Mean Squared Error (RMSE)
        rmse = np.sqrt(mse)

        # Mean Absolute Percentage Error (MAPE)
        # Avoid division by zero by adding a small epsilon to y_test_flat if necessary
        epsilon = 1e-10
        y_true_safe = np.where(y_true == 0, epsilon, y_true)
        mape = np.mean(np.abs((y_true - y_pred) / y_true_safe)) * 100

                # Append the metrics for the current time step to the list
        metrics_list.append({
            'Time Step': i + 1,
            'MAE': mae,
            'RMSE': rmse,
            'MAPE (%)': mape
        })

    # Create a DataFrame from the list of metrics
    metrics_df = pd.DataFrame(metrics_list)
    metrics_df.set_index('Time Step', inplace=True)

    return metrics_df

In [29]:
mean_metrics_list=[]
#for length in best_hyperparameters.keys():
for length in best_hyperparameters_improved_model.keys():
    logger.info(length)
    
    # Number of runs
    n_runs = 10
    
    # get the best hyperparameter of each length
    #hyperparams = best_hyperparameters[length]
    hyperparams = best_hyperparameters_improved_model[length]
    
    # Initialize lists to store mean metrics and all metrics from each run
    mean_metrics_rows = []
    df_all_metrics_list = []
    successful_runs = []  # To track runs with valid mse

    for run in range(1, n_runs + 1):
        logger.info(f"\n--- Run {run} ---")

        # Optionally set a unique seed for each run to ensure variability
        seed = run + 100  
        # Train the model
        model, mse = train_model(hyperparams, data_dict, length, seed=None)
        
        if not np.isnan(mse):  # Only proceed if mse is valid
            successful_runs.append(run)
        
            X_train = data_dict[length]['X_train']
            y_train = data_dict[length]['y_train']
            X_val = data_dict[length]['X_val']
            y_val = data_dict[length]['y_val']
            X_test = data_dict[length]['X_test']
            y_test = data_dict[length]['y_test']

            #get the true flow and predicted flow
            y_pred_train, y_obs_train = make_prediction(model, X_train, y_train)
            y_pred_val, y_obs_val = make_prediction(model, X_val, y_val)
            y_pred_test, y_obs_test = make_prediction(model, X_test, y_test)

            #calculate the evaluation metrics of each output step
            df_train = evaluation(y_pred_train, y_obs_train).add_suffix('_train')
            df_val = evaluation(y_pred_val, y_obs_val).add_suffix('_val')
            df_test = evaluation(y_pred_test, y_obs_test).add_suffix('_test')

            df_all_metrics = pd.concat([df_train, df_val, df_test], axis=1)
            df_all_metrics.index.name = length

            # Append df_all_metrics to the list
            df_all_metrics_list.append(df_all_metrics)

            # Calculate mean for all output step
            #mean_metrics = df_val.mean()
            mean_metrics = pd.concat([df_train.mean(), df_val.mean(), df_test.mean()])
            mean_metrics_row = pd.DataFrame(mean_metrics).T
            mean_metrics_row['MSE_val(loss)'] = mse
            mean_metrics_row['input_len'] = length
            #mean_metrics_row = mean_metrics_row[['input_len','MSE_val(loss)', 'MAE_val', 'RMSE_val', 'MAPE (%)_val']]
            mean_metrics_row = mean_metrics_row[['input_len', 'MSE_val(loss)', 'MAE_train', 'RMSE_train', 'MAPE (%)_train',
                                             'MAE_val', 'RMSE_val', 'MAPE (%)_val',
                                             'MAE_test', 'RMSE_test', 'MAPE (%)_test']]

            # Append to the list
            mean_metrics_rows.append(mean_metrics_row)
        else:
            logger.info(f"Run {run} has mse=NaN, skipping.")
            
    # Check if there are successful runs before proceeding
    if successful_runs:       
        # Concatenate all df_all_metrics into a single DataFrame with a new level for runs
        concatenated_all_metrics = pd.concat(df_all_metrics_list, keys=successful_runs, names=['Run', 'Time Step'])

        # Calculate the mean across runs for each metric and time step
        # This will group by 'Time Step' and calculate the mean of each metric across all runs
        aggregated_all_metrics_mean = concatenated_all_metrics.groupby('Time Step').mean()
        aggregated_all_metrics_mean.index.name = length

        logger.info("\n--- Aggregated Mean of All Metrics Across 10(Successful) Runs ---")
        # Convert DataFrame to string
        aggregated_all_metrics_mean_str = aggregated_all_metrics_mean.to_string(index=False)    
        # Log the DataFrame
        logger.info("\n" + aggregated_all_metrics_mean_str)
        display(aggregated_all_metrics_mean)
    else:
        logger.info(f"No successful runs for input length {length}.")

    # Check if mean metrics were calculated
    if mean_metrics_rows:        
        # After all runs, create a DataFrame of mean metrics
        mean_metrics_df = pd.concat(mean_metrics_rows, ignore_index=True)
        # Calculate the mean of each metric across the 10 runs
        final_mean_metrics = mean_metrics_df.mean()

        # Create a DataFrame for the final mean metrics
        final_mean_metrics_df = pd.DataFrame(final_mean_metrics).T

        mean_metrics_list.append(final_mean_metrics_df)
    else:
        logger.info(f"No mean metrics calculated for input length {length}.")

mean_metrics_df = pd.concat(mean_metrics_list).reset_index(drop=True)
logger.info("\n--- Final Mean Metrics Across 10 Runs ---")
# Convert DataFrame to string
mean_metrics_df_str = mean_metrics_df.to_string(index=False)    
# Log the DataFrame
logger.info("\n" + mean_metrics_df_str)
mean_metrics_df

24

--- Run 1 ---


/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 70: early stopping
Restoring model weights from the end of the best epoch: 50.
Validation loss: 0.00419

--- Run 2 ---


/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 71: early stopping
Restoring model weights from the end of the best epoch: 51.
Validation loss: 0.00427

--- Run 3 ---


...

/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 59: early stopping
Restoring model weights from the end of the best epoch: 39.
Validation loss: 0.00491

--- Aggregated Mean of All Metrics Across 10(Successful) Runs ---


,MAE_train,RMSE_train,MAPE (%)_train,MAE_val,RMSE_val,MAPE (%)_val,MAE_test,RMSE_test,MAPE (%)_test
504,,,,,,,,,
1,24.049886,36.330477,36.326167,22.327337,30.729567,31.093459,18.007031,23.264603,14.601704
2,26.964062,41.131655,42.772339,25.332972,35.558765,36.184221,20.470045,27.905475,16.594911
3,28.349413,43.189016,46.373284,27.408271,38.890000,40.246542,23.153510,30.190798,19.007819
4,28.951319,44.148226,49.223139,28.569073,40.844919,43.264576,24.793197,32.016939,20.189584
5,29.605911,45.276401,51.702777,29.453853,42.579398,45.136870,25.827114,32.895332,20.933869
6,30.967318,46.807123,56.571533,30.656695,44.317764,48.285117,25.909571,33.354290,20.908860



--- Final Mean Metrics Across 10 Runs ---


,input_len,MSE_val(loss),MAE_val,RMSE_val,MAPE (%)_val
0,24.0,0.004216,25.932460,36.001607,40.924680
1,48.0,0.004402,26.838355,36.811166,44.458924
2,72.0,0.004286,26.305974,36.313192,41.092844
3,96.0,0.004261,26.313900,36.210094,43.137760
4,120.0,0.004112,26.032372,35.535510,41.156442
5,144.0,0.004549,27.456148,37.332451,42.754682
6,168.0,0.004276,26.563577,36.209041,40.251372
7,192.0,0.004108,26.147941,35.547712,40.695259
8,216.0,0.003775,25.428305,34.071939,41.749625
9,240.0,0.003859,25.694276,34.457498,43.870160


In [28]:
mean_metrics_df

,input_len,MSE_val(loss),MAE_val,RMSE_val,MAPE (%)_val
0,24.0,0.004211,26.078336,36.003758,41.020640
1,48.0,0.004374,26.549048,36.693637,43.406696
2,72.0,0.004357,26.776325,36.609731,43.280924
3,96.0,0.004264,26.375986,36.260495,43.298351
4,120.0,0.004148,26.150845,35.685248,41.796633
5,144.0,0.004185,25.876769,35.851591,39.127578
6,168.0,0.004297,26.577252,36.321709,40.402273
7,192.0,0.004218,26.435064,35.987619,42.069822
8,216.0,0.003788,25.147041,34.144567,40.511893
9,240.0,0.003904,25.508670,34.659827,41.343390


# Code to predict next 6 steps simultaneously

#### We will use
* the last 12 steps

* previous one week (24 steps)

* previous one month  (168 steps)

*  to forecast current (0 step) and 5 steps ahead

In [11]:
# Create input-output sequences with the provided function
X_train, y_train, X_train_df, y_train_df = create_multi_step_sequence(train_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=6)
X_valid, y_valid, X_valid_df, y_valid_df = create_multi_step_sequence(valid_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=6)
X_test, y_test, X_test_df, y_test_df = create_multi_step_sequence(test_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=6)

ValueError: cannot reshape array of size 1141040 into shape (168,1)

## 5. Define LSTM model

In [12]:
def create_lstm_model_multi_step(input_shape, n_outputs, units, dropout_rate, learning_rate):
    model = keras.Sequential()
    model.add(LSTM(units=units, activation='tanh', input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(Dense(n_outputs))
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        loss='mse',  # Mean Squared Error loss for regression
        optimizer=optimizer,
        metrics=['mae']  # Mean Absolute Error as a metric
    )
    return model

input_shape = (x_train_scaled.shape[1], x_train_scaled.shape[2])
n_outputs = y_train.shape[1]

## 6. Set Up Hyperparameter Grid

In [17]:
# Hyperparameter options
units_list = [50, 100, 200]
dropout_rates = [0, 0.3, 0.5]
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [32, 64, 128]

# Create all possible combinations
hyperparameter_combinations = list(product(units_list, dropout_rates, learning_rates, batch_sizes))

## 7. Train the Model with Hyperparameter Tuning

In [14]:
# Initialize variables to store the best model and hyperparameters
best_val_mae = np.inf
best_hyperparams = None
best_model = None

for idx, (units, dropout_rate, learning_rate, batch_size) in enumerate(hyperparameter_combinations):
    print(f"\nCombination {idx+1}/{len(hyperparameter_combinations)}")
    print(f"Training with units={units}, dropout_rate={dropout_rate}, learning_rate={learning_rate}, batch_size={batch_size}")

    # Create the LSTM model with the current hyperparameters
    model = create_lstm_model_multi_step(
        input_shape=input_shape,
        n_outputs=n_outputs,
        units=units,
        dropout_rate=dropout_rate,
        learning_rate=learning_rate
    )

    # Initialize EarlyStopping
    early_stopping = EarlyStopping(
        monitor='val_mae',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )

    # Train the model
    history = model.fit(
        x_train_scaled, y_train_scaled,
        epochs=50,
        batch_size=batch_size,
        validation_data=(x_val_scaled, y_val_scaled),
        callbacks=[early_stopping],
        verbose=0
    )

    # Get the best validation MAE from this training run
    val_mae = min(history.history['val_mae'])
    print(f"Validation MAE: {val_mae:.4f}")

    # Update best model if current one is better
    if val_mae < best_val_mae:
        best_val_mae = val_mae
        best_model = model
        best_hyperparams = {
            'units': units,
            'dropout_rate': dropout_rate,
            'learning_rate': learning_rate,
            'batch_size': batch_size
        }

print("\nBest Hyperparameters:")
for param, value in best_hyperparams.items():
    print(f"{param}: {value}")
print(f"Best Validation MAE: {best_val_mae:.4f}")


Combination 1/81
Training with units=50, dropout_rate=0, learning_rate=0.01, batch_size=32


E:\program\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 43: early stopping
Restoring model weights from the end of the best epoch: 33.
Validation MAE: 0.0602

Combination 2/81
Training with units=50, dropout_rate=0, learning_rate=0.01, batch_size=64
Epoch 35: early stopping
Restoring model weights from the end of the best epoch: 25.
Validation MAE: 0.0600

Combination 3/81
Training with units=50, dropout_rate=0, learning_rate=0.01, batch_size=128
Epoch 35: early stopping
Restoring model weights from the end of the best epoch: 25.
Validation MAE: 0.0600

Combination 4/81
Training with units=50, dropout_rate=0, learning_rate=0.001, batch_size=32
Restoring model weights from the end of the best epoch: 50.
Validation MAE: 0.0618

Combination 5/81
Training with units=50, dropout_rate=0, learning_rate=0.001, batch_size=64
Restoring model weights from the end of the best epoch: 44.
Validation MAE: 0.0634

Combination 6/81
Training with units=50, dropout_rate=0, learning_rate=0.001, batch_size=128
Restoring model weights from the end of the b

Restoring model weights from the end of the best epoch: 50.
Validation MAE: 0.0708

Combination 46/81
Training with units=100, dropout_rate=0.5, learning_rate=0.01, batch_size=32
Epoch 27: early stopping
Restoring model weights from the end of the best epoch: 17.
Validation MAE: 0.0605

Combination 47/81
Training with units=100, dropout_rate=0.5, learning_rate=0.01, batch_size=64
Epoch 31: early stopping
Restoring model weights from the end of the best epoch: 21.
Validation MAE: 0.0606

Combination 48/81
Training with units=100, dropout_rate=0.5, learning_rate=0.01, batch_size=128
Epoch 37: early stopping
Restoring model weights from the end of the best epoch: 27.
Validation MAE: 0.0608

Combination 49/81
Training with units=100, dropout_rate=0.5, learning_rate=0.001, batch_size=32
Restoring model weights from the end of the best epoch: 50.
Validation MAE: 0.0616

Combination 50/81
Training with units=100, dropout_rate=0.5, learning_rate=0.001, batch_size=64
Restoring model weights fro

In [ ]:
13:33

Best Hyperparameters for full time:
units: 200
dropout_rate: 0.5
learning_rate: 0.01
batch_size: 32
Best Validation MAE: 0.0503

Best Hyperparameters for after covid:
units: 100
dropout_rate: 0.3
learning_rate: 0.01
batch_size: 128
Best Validation MAE: 0.0595

## 8. Make predictions

In [15]:
# Make predictions
y_pred_scaled = best_model.predict(x_test_scaled)

# Reshape for inverse scaling
y_pred_reshaped = y_pred_scaled.reshape(-1, 1)
y_test_reshaped = y_test_scaled.reshape(-1, 1)

# Inverse transform
y_pred_inverse = y_scaler.inverse_transform(y_pred_reshaped).reshape(n_test_samples, n_outputs)
y_test_inverse = y_scaler.inverse_transform(y_test_reshaped).reshape(n_test_samples, n_outputs)

y_test_flat = y_test_inverse.flatten()
y_pred_flat = y_pred_inverse.flatten()

66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


## 9. Evaluating the LSTM Model

In [16]:
# Compute Metrics for Each Time Step

for i in range(n_outputs):
    y_true = y_test_inverse[:, i]
    y_pred = y_pred_inverse[:, i]

    # Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_true, y_pred)

    # Mean Squared Error (MSE)
    mse = mean_squared_error(y_true, y_pred)

    # Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)

    # Mean Absolute Percentage Error (MAPE)
    # Avoid division by zero by adding a small epsilon to y_test_flat if necessary
    epsilon = 1e-10
    y_true_safe = np.where(y_true == 0, epsilon, y_true)
    mape = np.mean(np.abs((y_true - y_pred) / y_true_safe)) * 100

    print(f"\nTime Step {i+1} Evaluation Metrics:")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"MAPE: {mape:.2f}%")


Time Step 1 Evaluation Metrics:
RMSE: 28.4454
MAE: 20.8450
MAPE: 19.44%

Time Step 2 Evaluation Metrics:
RMSE: 36.2563
MAE: 26.7673
MAPE: 25.02%

Time Step 3 Evaluation Metrics:
RMSE: 43.7203
MAE: 33.1341
MAPE: 29.97%

Time Step 4 Evaluation Metrics:
RMSE: 49.3573
MAE: 38.0972
MAPE: 35.19%

Time Step 5 Evaluation Metrics:
RMSE: 53.2812
MAE: 41.5888
MAPE: 39.42%

Time Step 6 Evaluation Metrics:
RMSE: 53.3926
MAE: 42.0135
MAPE: 44.19%


# Code to predict next 6 steps step-by-step

#### We will use
* the last 12 steps

* previous one week (24 steps)

* previous one month  (168 steps)

*  to forecast current (0 step)

## 4. Create input and output data

In [29]:
# Create input-output sequences with the provided function
X_train, y_train, X_train_df, y_train_df = create_multi_step_sequence(train_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=1)
X_valid, y_valid, X_valid_df, y_valid_df = create_multi_step_sequence(valid_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=1)
X_test, y_test, X_test_df, y_test_df = create_multi_step_sequence(test_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=1)

In [42]:
X_train.shape, X_valid.shape, X_test.shape

((19570, 14, 1), (6907, 14, 1), (5623, 14, 1))

## 5. Normalise the data after split (step-by-step)

Normalise X

In [31]:
# Separate scalers for inputs and outputs
x_scaler = MinMaxScaler(feature_range=(0, 1))
y_scaler = MinMaxScaler(feature_range=(0, 1))

# Reshape x_train to 2D for scaling
n_samples, n_timesteps, n_features = X_train.shape
x_train_reshaped = X_train.reshape(-1, n_features)  # Shape: (n_samples * n_timesteps, n_features)
# Fit the scaler on the training data
x_scaler.fit(x_train_reshaped)
# Transform the training data
x_train_scaled = x_scaler.transform(x_train_reshaped)
# Reshape back to original shape
x_train_scaled = x_train_scaled.reshape(n_samples, n_timesteps, n_features)

# x_val
n_val_samples = X_valid.shape[0]
x_val_reshaped = X_valid.reshape(-1, n_features)
x_val_scaled = x_scaler.transform(x_val_reshaped)
x_val_scaled = x_val_scaled.reshape(n_val_samples, n_timesteps, n_features)

# x_test
n_test_samples = X_test.shape[0]
x_test_reshaped = X_test.reshape(-1, n_features)
x_test_scaled = x_scaler.transform(x_test_reshaped)
x_test_scaled = x_test_scaled.reshape(n_test_samples, n_timesteps, n_features)

Normalise y

In [32]:
# Reshape y_train to 2D for scaling
y_train_reshaped = y_train.reshape(-1, 1)  # Shape: (n_samples * n_outputs, 1)
# Fit the scaler on the training data
y_scaler.fit(y_train_reshaped)
# Transform the training data
y_train_scaled = y_scaler.transform(y_train_reshaped)
# Reshape back to original shape
y_train_scaled = y_train_scaled.reshape(n_samples, y_train.shape[1])

# y_val
y_val_reshaped = y_valid.reshape(-1, 1)
y_val_scaled = y_scaler.transform(y_val_reshaped)
y_val_scaled = y_val_scaled.reshape(n_val_samples, y_valid.shape[1])

# y_test
y_test_reshaped = y_test.reshape(-1, 1)
y_test_scaled = y_scaler.transform(y_test_reshaped)
y_test_scaled = y_test_scaled.reshape(n_test_samples, y_test.shape[1])

## 6. Build LSTM

In [21]:
def create_lstm_model(input_shape, units, dropout_rate, learning_rate):
    model = keras.Sequential()
    model.add(LSTM(units=units, activation='tanh', input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1)) # Output layer for one-step prediction
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        loss='mse',
        optimizer=optimizer,
        metrics=['mae']
    )
    return model

input_shape = (x_train_scaled.shape[1], x_train_scaled.shape[2])

## 7. Hyperparameter Tuning

In [22]:
# Hyperparameter options
units_list = [50, 100, 200]
dropout_rates = [0, 0.3, 0.5]
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [32, 64, 128]

# Create all possible combinations
hyperparameter_combinations = list(product(units_list, dropout_rates, learning_rates, batch_sizes))

In [23]:
# Initialize variables to store the best model and hyperparameters
best_val_mae = np.inf
best_hyperparams = None
best_model = None

for idx, (units, dropout_rate, learning_rate, batch_size) in enumerate(hyperparameter_combinations):
    print(f"\nCombination {idx+1}/{len(hyperparameter_combinations)}")
    print(f"Training with units={units}, dropout_rate={dropout_rate}, learning_rate={learning_rate}, batch_size={batch_size}")

    # Create the LSTM model with the current hyperparameters
    model = create_lstm_model(
        input_shape=input_shape,
        units=units,
        dropout_rate=dropout_rate,
        learning_rate=learning_rate
    )

    # Initialize EarlyStopping
    early_stopping = EarlyStopping(
        monitor='val_mae',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )

    # Train the model
    history = model.fit(
        x_train_scaled, y_train_scaled,
        epochs=50,
        batch_size=batch_size,
        validation_data=(x_val_scaled, y_val_scaled),
        callbacks=[early_stopping],
        verbose=0
    )

    # Get the best validation MAE from this training run
    val_mae = min(history.history['val_mae'])
    print(f"Validation MAE: {val_mae:.4f}")

    # Update best model if current one is better
    if val_mae < best_val_mae:
        best_val_mae = val_mae
        best_model = model
        best_hyperparams = {
            'units': units,
            'dropout_rate': dropout_rate,
            'learning_rate': learning_rate,
            'batch_size': batch_size
        }

print("\nBest Hyperparameters:")
for param, value in best_hyperparams.items():
    print(f"{param}: {value}")
print(f"Best Validation MAE: {best_val_mae:.4f}")



Combination 1/81
Training with units=50, dropout_rate=0, learning_rate=0.01, batch_size=32


E:\program\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 40: early stopping
Restoring model weights from the end of the best epoch: 30.
Validation MAE: 0.0452

Combination 2/81
Training with units=50, dropout_rate=0, learning_rate=0.01, batch_size=64
Epoch 44: early stopping
Restoring model weights from the end of the best epoch: 34.
Validation MAE: 0.0452

Combination 3/81
Training with units=50, dropout_rate=0, learning_rate=0.01, batch_size=128
Epoch 23: early stopping
Restoring model weights from the end of the best epoch: 13.
Validation MAE: 0.0466

Combination 4/81
Training with units=50, dropout_rate=0, learning_rate=0.001, batch_size=32
Restoring model weights from the end of the best epoch: 48.
Validation MAE: 0.0450

Combination 5/81
Training with units=50, dropout_rate=0, learning_rate=0.001, batch_size=64
Restoring model weights from the end of the best epoch: 49.
Validation MAE: 0.0463

Combination 6/81
Training with units=50, dropout_rate=0, learning_rate=0.001, batch_size=128
Restoring model weights from the end of the b

Restoring model weights from the end of the best epoch: 46.
Validation MAE: 0.0513

Combination 46/81
Training with units=100, dropout_rate=0.5, learning_rate=0.01, batch_size=32
Epoch 29: early stopping
Restoring model weights from the end of the best epoch: 19.
Validation MAE: 0.0466

Combination 47/81
Training with units=100, dropout_rate=0.5, learning_rate=0.01, batch_size=64
Epoch 25: early stopping
Restoring model weights from the end of the best epoch: 15.
Validation MAE: 0.0460

Combination 48/81
Training with units=100, dropout_rate=0.5, learning_rate=0.01, batch_size=128
Epoch 43: early stopping
Restoring model weights from the end of the best epoch: 33.
Validation MAE: 0.0458

Combination 49/81
Training with units=100, dropout_rate=0.5, learning_rate=0.001, batch_size=32
Restoring model weights from the end of the best epoch: 49.
Validation MAE: 0.0463

Combination 50/81
Training with units=100, dropout_rate=0.5, learning_rate=0.001, batch_size=64
Epoch 34: early stopping
Re

Best Hyperparameters full time:
units: 50
dropout_rate: 0
learning_rate: 0.01
batch_size: 128
Best Validation MAE: 0.0388

Best Hyperparameters after covid:
units: 100
dropout_rate: 0
learning_rate: 0.001
batch_size: 32
Best Validation MAE: 0.0450

#### Recreate the model¶

In [33]:
def best_lstm_model(input_shape, units, dropout_rate, learning_rate):
    model = keras.Sequential()
    model.add(LSTM(units=units, activation='tanh', input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1)) # Output layer for one-step prediction
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        loss='mse',
        optimizer=optimizer,
        metrics=['mae']
    )
    return model

In [34]:
input_shape = (x_train_scaled.shape[1], x_train_scaled.shape[2])
units=50
dropout_rate=0
learning_rate=0.01
batch_size=128

#### Retrain the model

In [35]:
# Initialize EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_mae',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

In [37]:
# Train the model
history = best_model.fit(
    x_train_scaled, y_train_scaled,
    epochs=50,  # You can adjust this as needed
    batch_size=batch_size,
    validation_data=(x_val_scaled, y_val_scaled),
    callbacks=[early_stopping],
    verbose=1  # Set to 1 to see detailed training output
)

Epoch 1/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0044 - mae: 0.0474 - val_loss: 0.0039 - val_mae: 0.0405
Epoch 2/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 0.0043 - mae: 0.0465 - val_loss: 0.0039 - val_mae: 0.0406
Epoch 3/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0043 - mae: 0.0467 - val_loss: 0.0040 - val_mae: 0.0432
Epoch 4/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0044 - mae: 0.0472 - val_loss: 0.0039 - val_mae: 0.0401
Epoch 5/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0043 - mae: 0.0469 - val_loss: 0.0040 - val_mae: 0.0430
Epoch 6/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0044 - mae: 0.0473 - val_loss: 0.0039 - val_mae: 0.0399
Epoch 7/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0043 - mae: 0.0464 - val_loss: 0.0039 - val_mae: 0.0406
Epoch 8/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 0.0045 - mae: 0.0475 - val_loss: 0.0039 - val_mae: 0.0416
Epoch 9/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/

## 8. Recursive Forecasting with LSTM (step-by-step)

In [38]:
def recursive_forecast(model, x_test_scaled, start_index, n_steps, x_scaler, y_scaler):
    """
    Perform recursive forecasting using the trained model.

    Parameters:
    - model: Trained model
    - x_test_scaled: The scaled test input data (shape: n_samples, n_timesteps, n_features)
    - start_index: The starting index in the test data
    - n_steps: Number of future steps to predict
    - x_scaler: Scaler used for input features
    - y_scaler: Scaler used for target variable

    Returns:
    - predictions: List of predicted values (in original scale)
    """
    predictions = []
    scaled_predictions = []
    current_input = x_test_scaled[start_index].copy()  # Shape: (n_timesteps, n_features)

    for step in range(n_steps):
        # Reshape to (1, n_timesteps, n_features) for prediction
        input_seq = current_input.reshape((1, current_input.shape[0], current_input.shape[1]))

        # Predict the next time step (scaled)
        yhat_scaled = model.predict(input_seq, verbose=0)  # Shape: (1, 1)

        # Inverse transform the prediction to original scale
        yhat = y_scaler.inverse_transform(yhat_scaled)[0, 0]
        
        # Transform the prediction back to input feature scale for lag features
        yhat_for_input = x_scaler.transform(yhat.reshape(-1, 1))[0, 0]

        # Append predictions
        predictions.append(yhat)
        scaled_predictions.append(yhat_for_input)

        # Move to the next time step in x_test_scaled
        next_index = start_index + step + 1
        if next_index < len(x_test_scaled):
            # Use features from the next time step
            next_input = x_test_scaled[next_index].copy()
        else:
            # Reached the end of x_test_scaled
            break

        # Update lag features with available scaled predictions
        for lag in range(1, min(step + 1, 6) + 1):
            feature_index = 12 - lag  # lag1 is at index 11
            next_input[feature_index, 0] = scaled_predictions[-lag]

        # Keep lag24 and lag168 as they are, or update if necessary

        # Set current_input for next iteration
        current_input = next_input

    return predictions

## 9. Make step-by-step prediction

In [39]:
# Number of steps to predict
n_steps = 6

# Initialize lists to store predictions and actual values
all_predictions = []
all_actuals = []

# Ensure we have enough data for recursive predictions
n_test_samples = x_test_scaled.shape[0]

for i in range(n_test_samples - n_steps):
    # Perform recursive forecasting
    predictions = recursive_forecast(
        model=best_model,
        x_test_scaled=x_test_scaled,
        start_index=i,
        n_steps=n_steps,
        x_scaler=x_scaler,
        y_scaler=y_scaler
    )

    # Get the actual future values (in original scale)
    actual_values = y_test[i+1:i + len(predictions) + 1].flatten()

    # Store the predictions and actual values
    all_predictions.append(predictions)
    all_actuals.append(actual_values)

# Convert lists to numpy arrays
all_predictions = np.array(all_predictions)
all_actuals = np.array(all_actuals)

## 10. Evaluating the LSTM Model

In [40]:
# Compute evaluation metrics
epsilon = 1e-10  # To avoid division by zero in MAPE

for i in range(n_steps):
    y_true = all_actuals[:, i]
    y_pred = all_predictions[:, i]

    # Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_true, y_pred)

    # Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    # Mean Absolute Percentage Error (MAPE)
    y_true_safe = np.where(y_true == 0, epsilon, y_true)
    mape = np.mean(np.abs((y_true - y_pred) / y_true_safe)) * 100

    print(f"\nTime Step {i+1} Evaluation Metrics:")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"MAPE: {mape:.2f}%")


Time Step 1 Evaluation Metrics:
RMSE: 41.4159
MAE: 29.8309
MAPE: 32.89%

Time Step 2 Evaluation Metrics:
RMSE: 44.7596
MAE: 31.9444
MAPE: 36.16%

Time Step 3 Evaluation Metrics:
RMSE: 47.2820
MAE: 33.4781
MAPE: 39.15%

Time Step 4 Evaluation Metrics:
RMSE: 49.5900
MAE: 34.8770
MAPE: 41.75%

Time Step 5 Evaluation Metrics:
RMSE: 51.5312
MAE: 35.9326
MAPE: 43.32%

Time Step 6 Evaluation Metrics:
RMSE: 53.2142
MAE: 36.6299
MAPE: 44.43%


In [ ]:
step-by-step full date